In [1]:
import pandas as pd
import seaborn as sb
import numpy as np
%matplotlib inline

import os
os.chdir('/home/admroot/boot')
print(os.getcwd())

import sys

sys.stdout = open('/home/admroot/avglog', 'w')

import datetime

from hyperopt import hp, fmin, tpe
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

import workdir.classes.config
from qml.cv import QCV
from qml.models import QXgb, QAvg, QRankedAvg, QRankedByLineAvg, QStackModel
from workdir.classes.models import qm
import numpy as np
from qml.helpers import get_engine
import random

/home/admroot/boot


/home/admroot/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
cv = QCV(qm)

In [5]:
_, conn = get_engine()
cv = QCV(qm)


conn.execute("update qml_models set level=3 where level=1 and cls in ('qavg', 'qrankedavg', 'QRankedByLineAvg', 'QStackModel')")
res = conn.execute(
    """
        select data_id, cls, descr, 
        substring_index(group_concat(model_id order by cv_score), ',', 50) as models
        from qml_results r 
        inner join qml_models m using(model_id) 
        where m.level<=2 and cv_score < 0.38
        group by data_id, cls, descr
    """
).fetchall()

results = []

for r in res:
    for m in r['models'].split(','):
        results.append([int(m), r['data_id']])

/usr/local/lib/python3.5/dist-packages/sqlalchemy/engine/default.py:470: Warning: (1260, 'Row 129 was cut by GROUP_CONCAT()')
  cursor.execute(statement, parameters)
/usr/local/lib/python3.5/dist-packages/sqlalchemy/engine/default.py:470: Warning: (1260, 'Row 258 was cut by GROUP_CONCAT()')
  cursor.execute(statement, parameters)


In [6]:
for i in range(3000):
    random.shuffle(results)
    models = list(results[:random.randint(2, 15)])
    models = sorted(models, key=lambda x: (x[0], x[1]))
    print(models)
    sys.stdout.flush()
    

    model_id = qm.add_by_params(
        QAvg(models)
    )
    print(model_id,cv.cross_val(model_id, -1))
    sys.stdout.flush()
    
    model_id = qm.add_by_params(
        QAvg(models, is_geom=True)
    )
    print(model_id,cv.cross_val(model_id, -1))
    sys.stdout.flush()
    
    model_id = qm.add_by_params(
        QRankedAvg(models)
    )
    print(model_id,cv.cross_val(model_id, -1))
    sys.stdout.flush()
    
    model_id = qm.add_by_params(
        QRankedByLineAvg(models)
    )
    print(model_id,cv.cross_val(model_id, -1))
    sys.stdout.flush()
    
#     model_id = qm.add_by_params(
#         QStackModel(models, 1)
#     )
#     print(model_id,cv.cross_val(model_id, -1))
#     sys.stdout.flush()
    
#     model_id = qm.add_by_params(
#         QStackModel(models, 2014)
#     )
#     print(model_id,cv.cross_val(model_id, -1))
#     sys.stdout.flush()
    
    #model_id = qm.add_by_params(
    #    QStackModel(models, 2127)
    #)
    #print(model_id,cv.cross_val(model_id, -1))
    sys.stdout.flush()
    
    conn.execute("update qml_models set level=3 where level=1 and cls in ('qavg', 'qrankedavg', 'QRankedByLineAvg', 'QStackModel')")
    
conn.execute("update qml_models set level=3 where level=1 and cls in ('qavg', 'qrankedavg', 'QRankedByLineAvg', 'QStackModel')")

OperationalError: (_mysql_exceptions.OperationalError) (1040, 'Too many connections')